# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import numpy as np
import pandas as pd
import requests
from pathlib import Path
from urllib import parse

# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mugia,43.1041,-9.2179,21.06,63,34,5.89,ES,1689418408
1,1,utrik,11.2278,169.8474,28.27,74,82,7.95,MH,1689418410
2,2,sitka,57.0531,-135.3300,12.94,94,100,2.06,US,1689418411
3,3,cova figueira,14.8905,-24.2934,24.12,80,99,0.93,CV,1689418413
4,4,blackmans bay,-43.0167,147.3167,9.29,63,34,6.18,AU,1689418414


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

def plotdots(df,attr):

    plot = df.hvplot.points(
        'Lng', 
        'Lat', 
        geo=True, 
        tiles='OSM', 
        color=attr, 
        size=attr, 
        hover_cols=['City'], 
        title=attr+' of Cities'
    )

    return plot

plot = plotdots(city_data_df,'Humidity')

plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_weather_df = (
  city_data_df
  [city_data_df['Max Temp']<=30]
  [city_data_df['Max Temp']>=20]
  [city_data_df['Humidity']<=70]
  [city_data_df['Cloudiness']<=80]
  .dropna()
)

ideal_weather_df.head()

/var/folders/df/p5245drj1ks2x4rkg5fmc4_40000gn/T/ipykernel_15149/2416464247.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  city_data_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mugia,43.1041,-9.2179,21.06,63,34,5.89,ES,1689418408
15,15,newman,37.3138,-121.0208,22.99,52,0,1.67,US,1689418428
18,18,sinop,41.7711,34.8709,24.61,61,1,5.60,TR,1689418433
27,27,beloyarskiy,63.7119,66.6722,23.98,30,5,4.51,RU,1689418286
30,30,svetlyy yar,48.4748,44.7810,22.78,45,78,9.06,RU,1689418449


In [15]:
plot = plotdots(ideal_weather_df,'Humidity')

plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City','Country','Lat','Lng','Humidity']].reset_index()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

hotel_df.head()


,index,City,Country,Lat,Lng,Humidity,Hotel Name
0,0,mugia,ES,43.1041,-9.2179,63,
1,15,newman,US,37.3138,-121.0208,52,
2,18,sinop,TR,41.7711,34.8709,61,
3,27,beloyarskiy,RU,63.7119,66.6722,30,
4,30,svetlyy yar,RU,48.4748,44.7810,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation',
    'limit': 1,
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary and convert to JSON
    query_url = base_url+'?'+parse.urlencode(params,safe=',:')

    try:
        response = requests.get(query_url)
        response_json = response.json()['features'][0]['properties']
        hotel_name = response_json['name']

    except (KeyError, IndexError):
        hotel_name = 'No hotel found'

    hotel_df['Hotel Name'][index] = hotel_name
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


/var/folders/df/p5245drj1ks2x4rkg5fmc4_40000gn/T/ipykernel_15149/3722386831.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'][index] = hotel_name


mugia - nearest hotel: Hábitat
newman - nearest hotel: Hamlet Motel
sinop - nearest hotel: No hotel found
beloyarskiy - nearest hotel: Сибирь
svetlyy yar - nearest hotel: Рубеж
lamu - nearest hotel: Hotel Archipelago villa bed and breakfast
montepuez - nearest hotel: Complexo Messalo
kostomuksha - nearest hotel: Гостевой дом на Брусничной
kalengwa - nearest hotel: No hotel found
fort-shevchenko - nearest hotel: Akku
hadibu - nearest hotel: No hotel found
benguela - nearest hotel: Hotel Moibela
tolanaro - nearest hotel: Hôtel Mahavokey
california city - nearest hotel: Best Western California City Inn & Suites
west island - nearest hotel: My Island Home
fatezh - nearest hotel: Фатеж
oulad teima - nearest hotel: Riad El Koudia
kruisfontein - nearest hotel: Du Plessis Self Catering
namibe - nearest hotel: Pensão Delta do Deserto
east london - nearest hotel: No hotel found
cadale - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
frontera - nearest hotel: Salón 

,index,City,Country,Lat,Lng,Humidity,Hotel Name
0,0,mugia,ES,43.1041,-9.2179,63,Hábitat
1,15,newman,US,37.3138,-121.0208,52,Hamlet Motel
2,18,sinop,TR,41.7711,34.8709,61,No hotel found
3,27,beloyarskiy,RU,63.7119,66.6722,30,Сибирь
4,30,svetlyy yar,RU,48.4748,44.7810,45,Рубеж
...,...,...,...,...,...,...,...
111,643,fernley,US,39.6080,-119.2518,17,Lazy Inn
112,652,valozhyn,BY,54.0909,26.5225,42,Карона
113,662,bar,UA,49.0772,27.6826,39,No hotel found
114,668,mozarlandia,BR,-14.7447,-50.5706,60,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

hotel_df['has_hotel'] = np.where(hotel_df['Hotel Name']!='No hotel found', 1, 0)

plot = hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    tiles='OSM', 
    color='has_hotel',
    alpha = 0.8,
    hover_cols=['City','Hotel Name','Country'], 
    title='Hotels in ideal weather cities'
)


plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (has_hotel,City,Hotel Name,Country)